In [51]:
import pandas as pd
import numpy as np

In [52]:
lower_file = '~/Clubs/Launch/Zephyr/FSW.git/app/other/airbraker/data/omen_quantile_luts/lower_bounds.csv'
upper_file ='~/Clubs/Launch/Zephyr/FSW.git/app/other/airbraker/data/omen_quantile_luts/upper_bounds.csv'


In [53]:
lower = pd.read_csv(lower_file)
upper = pd.read_csv(upper_file)

In [54]:
indices_agree = np,all(lower.alt == upper.alt)
if not indices_agree:
    print("Baseline assumption failed. indices dont agree")

In [55]:
minVal = lower.alt.iloc[0]
maxVal = lower.alt.iloc[-1]
num = len(lower.alt)

In [56]:
print(f"Bounds: {minVal} to {maxVal} with {num} samples")

Bounds: 1200.0 to 3300.0 with 100 samples


In [ ]:
def gen_index(val, lo, hi, num_els):
    if (val < lo): return 0, 0
    if (val > hi): return num_els - 1, 0

    find = (num-1) * (val - lo) / (hi-lo)

    whole =  int(find)
    frac = find-whole
    if (whole >= num_els -1):
        return hi-1, 0
    
    return whole, frac

def lerp(amt, down, up):
    return down * (1-amt) + up * amt

def gen_both(whole, frac, lowers, uppers):
    downL, upL = lowers[whole], lowers[whole+1]
    downU, upU = uppers[whole], uppers[whole+1]
    return lerp(frac, downL, upL), lerp(frac, downU, upU)

def Q(z_hat_min, z_hat_max, z_hat):
    val = (z_hat - z_hat_min)/ (z_hat_max - z_hat_min)
    if (val > 1): return 1
    if val < 0: return 0
    return val

In [125]:
print('z_hat, z_dot_hat, z_hat_min, z_hat_max, effort')
for z_dot, z_dot_hat in [(1300, 240), (1900, 190), (1900, 180), (2200, 155), (2900, 90)]:
    i, f = gen_index(z_dot, minVal, maxVal, num)
    z_min, z_max = gen_both(i, f, lower.vel, upper.vel)
    q = Q(z_min, z_max, z_dot_hat)
    print(f"{z_dot}, {z_dot_hat}, {z_min}, {z_max}, {q}")

z_hat, z_dot_hat, z_hat_min, z_hat_max, effort
1300, 240, 221.06191285714283, 262.7005742857143, 0.45481978750311836
1900, 190, 177.95167, 198.74508, 0.5794302137071311
1900, 180, 177.95167, 198.74508, 0.09850861402723235
2200, 155, 155.05803428571429, 168.54802142857142, 0
2900, 90, 90.21191900000001, 92.71914471428573, 0
